In [1]:
from langchain.embeddings import OpenAIEmbeddings
from numpy import dot
from numpy.linalg import norm

In [3]:
#エンベッディングのモデル指定
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#クエリーをベクトル化（１）
query_vecter = embeddings.embed_query("飛行車の最高速度は？")

#ベクトル変換された値を表示
print(f"ベクトル化された質問:{query_vecter[:5]}")

#違う文字列をベクトル化（２）
document_1_vector = embeddings.embed_query("飛行車の最高速度は時速150キロメートルです。")
#違う文字列をベクトル化（３）
document_2_vector = embeddings.embed_query("鶏肉和適切に下味をつけた後、中火で焼きながらたまに裏返し、 \
外側は香ばしく柔らかく仕上げる。")

#（１）と（２）の比較
cos_sim_1 = dot(query_vecter, document_1_vector)/(norm(query_vecter)*norm(document_1_vector))
print(f"ドキュメント１と質問の類似度:{cos_sim_1}")
#（２）と（３）の比較
cos_sim_2 = dot(query_vecter, document_2_vector)/(norm(query_vecter)*norm(document_2_vector))
print(f"ドキュメント２と質問の類似度:{cos_sim_2}")

ベクトル化された質問:[-0.0006812680055983601, 0.00824013646957589, -0.004933948253756882, -0.021167207898880762, -0.008620157996529503]
ドキュメント１と質問の類似度:0.9275586408305281
ドキュメント２と質問の類似度:0.7388375742485174


# 与えられたPDFを元に回答

In [5]:
!pip install pymupdf

In [4]:
from langchain.document_loaders import PyMuPDFLoader

In [5]:
#PDFファイルを読み込む
loader = PyMuPDFLoader("IT用語100選.pdf")
documents = loader.load()
#ドキュメント数表示
print(f"ドキュメントの数:{len(documents)}")
#一つ目のドキュメントの内容表示
print(f"1つ目のドキュメントの内容:{documents[0].page_content}")
#一つ目のドキュメントのメタデータ表示
print(f"1つ目のドキュメントのメタデータ:{documents[0].metadata}")

ドキュメントの数:615
1つ目のドキュメントの内容:IT用語100選
非エンジニア
のための

1つ目のドキュメントのメタデータ:{'source': 'IT用語100選.pdf', 'file_path': 'IT用語100選.pdf', 'page': 0, 'total_pages': 615, 'format': 'PDF 1.7', 'title': 'PowerPoint プレゼンテーション', 'author': '石澤 衛', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2021', 'producer': 'Microsoft® PowerPoint® 2021', 'creationDate': "D:20230201135221+09'00'", 'modDate': "D:20230201135221+09'00'", 'trapped': ''}


In [9]:
!pip install spacy

   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ------------ --------------------------- 3.8/12.1 MB 123.5 MB/s eta 0:00:01
   ------------------------------- -------- 9.6/12.1 MB 122.6 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.1 MB 108.8 MB/s eta 0:00:01
   ---------------------------------------- 12.1/12.1 MB 81.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/182.0 kB ? eta -:--:--
   ---------------------------------------- 182.0/182.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/122.3 kB ? eta -:--:--
   ---------------------------------------- 122.3/122.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/479.7 kB ? eta -:--:--
   --------------------------------------- 479.7/479.7 kB 29.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 91.9 MB/s eta 0:00:00
   -------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.2 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
fastapi-cli 0.0.2 requires uvicorn[standard]>=0.29.0, but you have uvicorn 0.25.0 which is incompatible.


In [11]:
!python -m spacy download ja_core_news_sm

     ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
     --------------------------------------- 0.0/12.1 MB 330.3 kB/s eta 0:00:37
     --------------------------------------- 0.1/12.1 MB 525.1 kB/s eta 0:00:23
     - -------------------------------------- 0.4/12.1 MB 2.5 MB/s eta 0:00:05
     ----------- ---------------------------- 3.4/12.1 MB 19.6 MB/s eta 0:00:01
     ----------------------------- ---------- 9.0/12.1 MB 38.6 MB/s eta 0:00:01
     -------------------------------------  12.1/12.1 MB 108.8 MB/s eta 0:00:01
     -------------------------------------  12.1/12.1 MB 108.8 MB/s eta 0:00:01
     --------------------------------------- 12.1/12.1 MB 59.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB 435.7 kB/s eta 0:02:46
   ---------------------------------------- 0.4/72.0 MB 3.7 MB/s eta 0

In [6]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter

In [7]:
#
loader = PyMuPDFLoader("./sample.pdf")
documents = loader.load()
#
text_splitter=SpacyTextSplitter(
    chunk_size=300,
    pipeline="ja_core_news_sm"
)
#
splitted_documents = text_splitter.split_documents(documents)
#
print(f"分割前のドキュメント数:{len(documents)}")
print(f"分割後のドキュメント数:{len(splitted_documents)}")

分割前のドキュメント数:12
分割後のドキュメント数:56


In [8]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma

In [9]:
#
loader = PyMuPDFLoader("./sample.pdf")
documents = loader.load()
#
text_splitter=SpacyTextSplitter(
    chunk_size=300,
    pipeline="ja_core_news_sm"
)
#
splitted_documents = text_splitter.split_documents(documents)
#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)
#
database.add_documents(
    splitted_documents,
)
#
print("データベースの作成が完了しました！")

データベースの作成が完了しました！


In [10]:
#
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)
#
documents = database.similarity_search("飛行車の最高速度は？")
#
print(f"ドキュメント数:{len(documents)}")
#
for document in documents:
    #
    print(f"ドキュメントの内容:{document.page_content}")

ドキュメント数:4
ドキュメントの内容:飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容:飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容:第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。




第4条（特例の速度制限）
1.
緊急車両、公的機関の車両、及び関連する公的任務を遂行する車両については、第3条
の制限を超える速度での飛行が許可される。


2.
第1項の特例を適用する場合、事前または事後に関連する機関への届け出が必要とす
る。
ドキュメントの内容:第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)

#
query = "飛行車の最高速度は？"
#
documents = database.similarity_search(query)
#
documents_string = ""
#
for document in documents:
    documents_string += f"""
    ------------------------
    {document.page_content}
    """
#
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

#
result = chat([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])
#
print(result.content)

都市部においては時速150キロメートル、都市部以外の地域では時速250キロメートルです。


## チャット画面を作成する

In [ ]:
!pip install chainlit

### 手順
1.↓以下のセルをコピーしpyファイルを作る

2.chainlit run [Pythonファイル名]

In [ ]:
#理由は不明chainlitのターミナルでインストールするように言われる
!pip install langchain-community

In [ ]:
#
import chainlit as cl

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="準備ができました！メッセージを入力してください！").send()

@cl.on_message
async def on_message(input_message):
    print("入力されたメッセージ:" + input_message)
    await cl.Message(content="こんにちわ！").send()

In [ ]:
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)
#
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="準備ができました！メッセージを入力してください！").send()

@cl.on_message
async def on_message(input_message):
    #print("入力されたメッセージ:" + input_message)
    documents = database.similarity_search(input_message)
    #
    documents_string = ""
    #
    for document in documents:
        documents_string += f"""
        ------------------------
        {document.page_content}
        """
    #
    result = chat([
        HumanMessage(content=prompt.format(document=documents_string, query=input_message))
    ])    
    await cl.Message(content=result.content).send()  